In [36]:
import pysam
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import defaultdict

In [37]:
duplex_path = "test_duplex.bam"

output_path = "test_duplex_output.bed"

In [38]:
with pysam.AlignmentFile(duplex_path, "rb") as bam:

    strand_convert = {0:'+', 1:'-'}
    strand_to_color = {0:'255,182,193', 1:'100,149,237'}

    bed12_entries = []

    # Iterate over alignments
    for read in bam:

        # dict to hold modification type (ex m/a)
        mod_dict = defaultdict(lambda: defaultdict(list))

        if read.modified_bases_forward:
            for mod in read.modified_bases.keys(): # through the different types of mods
                mod_type = mod[2]
                strand = mod[1]
                print()
                for mod_site in read.modified_bases[mod]: # through each mod_base_forward
                    if mod_site[1] > 204 and mod_site[0] < read.query_alignment_end and mod_site[0] > read.query_alignment_start: # ML threshold
                        mod_dict[mod_type][strand].append( mod_site[0] - read.query_alignment_start )
                        #print( read.query_name, mod_type, strand )

            
            for side in mod_dict['m']:
                bed12_entry = ""
                bed12_entry += "\t".join([str(read.reference_name), str(read.query_alignment_start), str(read.query_alignment_end), 
                                          str(read.query_name), "0", str(strand_convert[side]), 
                                          str(read.query_alignment_start), str(read.query_alignment_end), str(strand_to_color[side]),
                                          str(len(mod_dict['m'][side])+2), "0," + "1,"*(len(mod_dict['m'][side])) + '1', '0,' + ','.join(map(str, sorted(mod_dict['m'][side]))) + f',{str(read.query_alignment_end-1)}' ]) + "\n"
                print( len(mod_dict['m'][side]) )
                bed12_entries.append( bed12_entry )
                

In [39]:
#str(len(mod_dict['m'][side])+2), "0," +"1,"*len(mod_dict['m'][side]) + "1,", f'{str(read.query_alignment_start)},' + ','.join(map(str, sorted(mod_dict['m'][side]))) + f',{str(read.query_alignment_end-2)}' ]) + "\n"

In [40]:
with open(output_path, "w") as file:
    for line in bed12_entries:
        file.write(line)
        print( line )

    print(f"The string has been written to {output_path}")

The string has been written to test_duplex_output.bed
